In [1]:
# set this path to learnMSA 
learn_msa_path = "../../learnMSA"

# !pip install tensorflow==2.10.1
# !pip install tensorflow_probability==0.18.0
# !pip install transformers
import sys, os

# os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.append(learn_msa_path) 
sys.path.append("../../bin")
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model

2024-07-20 08:13:08.768505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-20 08:13:08.837031: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-20 08:13:09.135135: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/conda/envs/cuda-11.2/lib
2024-07-20 08:13:09.135305: W tensorflow/com

In [2]:
batch_size = 2
seq_len = 500004
strand = '+'

emb = False
hmm_factor = 1
hmm_parallel = 817
trans_lstm = False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

model_path = "../../weights/tiberius_weights"
genome_path = f'{inp_data_dir}/genome.fa'
# output gtf file
gtf_out = 'tiberius.py'

pred_gtf = PredictionGTF( 
    model_path=model_path,
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    softmask=True, strand=strand,
    parallel_factor=hmm_parallel
)

In [3]:
# load model
pred_gtf.load_model()

# load input data x_seq 
x_seq, y_seq, coords = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True
)

Running gene pred hmm layer with parallel factor 817
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, None, 6)]    0           []                               
                                                                                                  
 initial_conv (Conv1D)          (None, None, 128)    2432        ['main_input[0][0]']             
                                                                                                  
 layer_normalization1 (LayerNor  (None, None, 128)   256         ['initial_conv[0][0]']           
 malization)                                                                                      
                                                                                                  
 conv_1 (Conv1D)                (None, 

In [4]:
# generate LSTM and HMM predictions
hmm_pred = pred_gtf.get_predictions(x_seq, hmm_filter=True)

# infer gene structures and write GTF file
pred_gtf.create_gtf(y_label=hmm_pred, coords=coords,
        out_file=gtf_out, f_chunks=x_seq, strand=strand)

### LSTM prediction


2024-07-20 08:13:17.082403: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2024-07-20 08:13:17.554684: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-07-20 08:13:17.554710: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2024-07-20 08:13:17.554798: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-07-20 08:13:18.154053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


LSTM took 1.1954469641049703 minutes to execute.
### HMM Viterbi
HMM took 0.5480005145072937 minutes to execute.
### LSTM prediction
LSTM took 0.2399880290031433 minutes to execute.
### HMM Viterbi
HMM took 0.05577863454818725 minutes to execute.


(<genome_anno.Anno at 0x7f3c845e7df0>, 209)